<a href="https://colab.research.google.com/github/elangbijak4/Riset-Smart-City/blob/main/Demo_AI_Driven_(Kasus_MLP2)_untuk_EMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# 1. Simulasi dataset kendaraan
def generate_data(n=2000):
    data = []
    for _ in range(n):
        speed = np.random.randint(0, 100)  # km/jam
        slope = np.random.choice(['flat', 'uphill', 'downhill'])
        battery = np.round(np.random.uniform(0, 1), 2)  # 0.0 - 1.0

        # Aturan keputusan simulatif
        if slope == 'uphill' and battery < 0.3:
            action = 'engine'
        elif speed < 40 and battery > 0.5:
            action = 'battery'
        else:
            action = np.random.choice(['battery', 'engine'])

        data.append([speed, slope, battery, action])

    return pd.DataFrame(data, columns=['speed', 'slope', 'battery', 'action'])

df = generate_data()

# 2. Pra-proses
df = pd.get_dummies(df, columns=['slope'])  # one-hot encoding
label_encoder = LabelEncoder()
df['action'] = label_encoder.fit_transform(df['action'])  # battery=0, engine=1

# Split dataset
# Ensure all features are float before converting to numpy array
X = df.drop('action', axis=1).astype(float).values
y = tf.keras.utils.to_categorical(df['action'].values, num_classes=2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Buat MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # 2 kelas: battery atau engine
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 4. Training
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.1)

# Evaluasi
loss, acc = model.evaluate(X_test, y_test)
print(f"\n🎯 Akurasi data uji: {acc:.2f}")

# 5. Fungsi prediksi untuk input baru
def predict_action(speed, slope, battery):
    slopes = ['slope_downhill', 'slope_flat', 'slope_uphill']
    slope_encoding = [1 if f"slope_{slope}" == s else 0 for s in slopes]
    # Ensure the input data for prediction is also in the correct numerical format
    input_data = np.array([[speed, battery] + slope_encoding], dtype=np.float32)
    pred = model.predict(input_data)
    pred_label = np.argmax(pred)
    return label_encoder.inverse_transform([pred_label])[0]

# 6. Tes prediksi
print("🔍 Prediksi aksi untuk speed=30, slope='flat', battery=0.6:",
      predict_action(30, 'flat', 0.6))
print("🔍 Prediksi aksi untuk speed=70, slope='uphill', battery=0.2:",
      predict_action(70, 'uphill', 0.2))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,530 (9.88 KB)

 Trainable params: 2,530 (9.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5308 - loss: 0.9011 - val_accuracy: 0.6938 - val_loss: 0.6572
Epoch 2/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6032 - loss: 0.6694 - val_accuracy: 0.5500 - val_loss: 0.7113
Epoch 3/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6010 - loss: 0.6857 - val_accuracy: 0.6875 - val_loss: 0.6203
Epoch 4/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6447 - loss: 0.6330 - val_accuracy: 0.6812 - val_loss: 0.5952
Epoch 5/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6290 - loss: 0.6461 - val_accuracy: 0.6187 - val_loss: 0.6855
Epoch 6/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6019 - loss: 0.6666 - val_accuracy: 0.7063 - val_loss: 0.5913
Epoch 7/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6345 - loss: 0.6252 - val_accuracy: 0.6500 - val_loss: 0.6237
Epoch 8/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6280 - loss: 0.6420 - val_accuracy: 0.7125 - val_loss